
# Gen‑AI Workshop Notebook: RAG → Workflows → Agents (Java Clean Architecture)

**Goal:** Apply Retrieval Augmented Generation (RAG), LangChain Workflows, and Agents to detect misplaced business logic in Java code, using a small Clean Architecture knowledge base.

**You will do (with minimal typing):**
- Use a ready‑made FAISS index over architecture rules
- Implement 1–2 line functions in marked code frames
- Run usage checks that apply your code to Java samples
- See an **expected output** box before each check

**Structure per exercise:**  
**Task → Coding frame (TODOs) → Model solution → Usage check (with expected output box).**

**Prereqs (one time):**
- Python 3.10+
- Packages will be installed below
- Set `OPENAI_API_KEY` (Colab: `from google.colab import userdata; os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")`)

**Time guidance:** RAG (25–30 min) → Workflows (20–25 min) → Agents (20–25 min) → MCP (optional, 10 min)



## Setup (run once at the top)

- Install dependencies
- Import libraries
- Load a small Clean Architecture rule base
- Build FAISS index (or reuse it if already built)
- Provide Java sample snippets
- Do a quick sanity check


In [ ]:

# Install (feel free to skip reruns if already installed)
# If running in environments without internet/package access, keep this cell for your local/Colab run.
try:
    import sentence_transformers, faiss, openai, langchain
except Exception:
    %pip install -q sentence-transformers faiss-cpu openai langchain
    
import os
import json
import random
import numpy as np
from typing import List
from dataclasses import dataclass

# LLM
from openai import OpenAI

# Embeddings
from sentence_transformers import SentenceTransformer

# Vector search
import faiss

# LangChain (chains and agents)
from langchain.chains import LLMChain, SequentialChain, TransformChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI  # thin wrapper over OpenAI client

# Misc
import textwrap

# ---- Reproducibility ----
random.seed(42)
np.random.seed(42)

# ---- API key expectation ----
if not os.environ.get("OPENAI_API_KEY"):
    print("⚠ Please set OPENAI_API_KEY. In Colab:")
    print("from google.colab import userdata; os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')")

# ---- Tiny Clean Architecture 'rule base' (demo size) ----
# In a real workshop you would load these from files. Here we inline minimal but representative text chunks.
chunks: List[str] = [
    "Controllers should delegate business rules to application or domain services; controllers handle HTTP, routing, and validation only.",
    "Repositories are responsible for persistence concerns only. They should not contain business rules or calculations.",
    "Entities should encapsulate core domain invariants but must not perform IO or depend on infrastructure concerns.",
    "Business rules should be placed in domain services or use cases. Keep side-effects and persistence out of the domain logic.",
    "Cross-cutting concerns like logging, metrics, and auth should be applied via decorators, interceptors, or middleware, not inside domain methods."
]

# ---- Build FAISS index ----
EMBED_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(EMBED_MODEL_NAME)

emb = model.encode(chunks, convert_to_numpy=True)
d = emb.shape[1]
index = faiss.IndexFlatIP(d)
# Normalize for cosine similarity via inner product
def _normalize(x: np.ndarray) -> np.ndarray:
    n = np.linalg.norm(x, axis=1, keepdims=True) + 1e-12
    return x / n

emb_n = _normalize(emb.copy())
index.add(emb_n)

# ---- Demo Java code samples ----
LEAKY_SAMPLES = {
    "order_controller": """
package com.example.leakydemo.controller;

public class OrderController {
    // Age check and discount calculation incorrectly in controller (business logic)
    public String createOrder(User user, OrderRequest req) {
        if (user.getAge() < 18) {  // business rule here (violation)
            return "Rejected: underage";
        }
        double discount = 0.0;      // business rule here (violation)
        if (req.getTotal() > 100) {
            discount = 0.1;
        }
        // ... persists via service later but logic sits here
        return "OK";
    }
}
""",

    "order_repository": """
package com.example.leakydemo.repository;

public class OrderRepository {
    // Persistence layer should not calculate discounts (violation)
    public double saveAndReturnDiscount(double total) {
        double discount = 0.0; // business logic (violation)
        if (total > 100) {
            discount = 0.1;
        }
        // ... pretend saving to DB ...
        return discount;
    }
}
""",

    "order_entity": """
package com.example.leakydemo.domain;

public class Order {
    private double total;
    public Order(double total) {
        this.total = total;
    }

    // Entities should not perform IO or call repositories (violation)
    public void ensurePersisted(OrderRepository repo) {
        repo.save(this); // IO/infrastructure from entity (violation)
    }
}
"""
}

def preview(text: str, n=400):
    return (text or "")[:n] + ("..." if text and len(text) > n else "")

print("✅ Setup complete. FAISS index built over", len(chunks), "rule chunks.")
print("✅ Available Java samples:", ", ".join(LEAKY_SAMPLES.keys()))



---

## Section 1 — Retrieval Augmented Generation (RAG)

We implement semantic retrieval over Clean Architecture rules and then augment an LLM analysis with the retrieved context.



### Exercise 1.1 — Implement Semantic Retrieval

**Your Task:** Complete the `retrieve_relevant_rules()` function to perform semantic search on the Clean Architecture knowledge base.

**What you need to implement:**
1. Encode the query using the sentence transformer model  
2. Search the FAISS index for top‑k most similar chunks  
3. Return the concatenated text of retrieved chunks

**Success Criteria:**
- Function returns a non‑empty string  
- Retrieved text contains relevant architecture keywords  
- Usage check shows controller/service/domain keywords


In [ ]:

def retrieve_relevant_rules(query: str, top_k: int = 3) -> str:
    """TODO: Implement semantic retrieval over rule chunks.
    Steps:
      1) query_embedding = model.encode([query], convert_to_numpy=True)
      2) normalize and search via FAISS (inner product)
      3) collect chunks by indices and join
    """
    # TODO: YOUR CODE HERE (minimal)
    # hint: use _normalize and index.search
    return ""  # placeholder



**Model solution (reference implementation):**


In [ ]:

def retrieve_relevant_rules(query: str, top_k: int = 3) -> str:
    q = model.encode([query], convert_to_numpy=True)
    qn = _normalize(q.astype(np.float32))
    scores, idx = index.search(qn, top_k)
    picked = [chunks[i] for i in idx[0] if i >= 0]
    text = "\n\n".join(picked)
    return text.replace("\u200b", "").replace("\ufeff", "")



**Expected output (roughly):**
- Contains terms like *controller*, *service*, *domain*, *repositories*
- Length clearly non‑empty (> 100 chars)
- 2–3 distinct text blocks joined

Now run the usage check:


In [ ]:

test_query = "business logic in controller"
retrieved = retrieve_relevant_rules(test_query)
print("=== Retrieved rules (preview) ===\n")
print(preview(retrieved, 500))
print("\nLength:", len(retrieved))



### Exercise 1.2 — Augment LLM with Retrieved Context

**Your Task:** Create a prompt that combines retrieved rules with Java code for violation analysis. Prompt text is **already provided**; your job is to call the API and return the response.

**What you need to implement:**
1. Use the provided **system** and **user** prompts
2. Call OpenAI Chat Completions
3. Return the response text

**Success Criteria:**
- LLM identifies at least one violation
- Response references specific architecture rules


In [ ]:

SYSTEM_PROMPT = (
    "You are a senior Java Clean Architecture reviewer. "
    "Identify layer violations succinctly and recommend concrete fixes."
)

def _build_user_prompt(java_code: str, rules: str) -> str:
    return f"""
[Task] Analyze the Java code for Clean Architecture violations.
[Rules]
{rules}

[Code]
{java_code}

[Output format]
- Violations (bullet points)
- Referenced rules (bullet points)
- Fix recommendation (1-2 sentences)
"""

def analyze_with_rag(java_code: str, rules: str) -> str:
    """TODO: Call OpenAI with given prompts and return the text response."""
    client = OpenAI()  # expects OPENAI_API_KEY in env
    # TODO: YOUR CODE HERE (2–3 lines to call chat.completions)
    return ""  # placeholder



**Model solution (reference implementation):**


In [ ]:

def analyze_with_rag(java_code: str, rules: str) -> str:
    client = OpenAI()
    user_prompt = _build_user_prompt(java_code, rules)
    resp = client.chat.completions.create(
        model="gpt-4.1-nano",
        temperature=0,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ]
    )
    return resp.choices[0].message.content.strip()



**Expected output (roughly):**
- Mentions at least one violation like *age check in controller* or *discount calculation in controller*
- References rules (controller → delegate; business rules → domain/services)
- Provides a concrete fix (move logic to service/domain)

Now run the usage check:


In [ ]:

sample_code = LEAKY_SAMPLES['order_controller']
rules = retrieve_relevant_rules(sample_code)
analysis = analyze_with_rag(sample_code, rules)
print("=== RAG-based analysis ===\n")
print(analysis)



---

## Section 2 — Workflows with LangChain (TransformChain → LLMChain → SequentialChain)



### Exercise 2.1 — Build a Sequential Chain

**Your Task:** Create a `SequentialChain` that connects retrieval and analysis steps.

**What you need to implement:**
1. Define `TransformChain` for retrieval (wraps `retrieve_relevant_rules`)  
2. Define `LLMChain` for analysis (prompt is provided)  
3. Connect them with `SequentialChain`

**Success Criteria:**
- Workflow executes without errors
- Output contains a violation analysis


In [ ]:

# Step 1 — Retrieval TransformChain
def transform_retrieval(inputs: dict) -> dict:
    """TODO: Use retrieve_relevant_rules on inputs['code'] and return {'rules': ...}"""
    # TODO: YOUR CODE HERE (1 line)
    return {"rules": ""}

retrieval_chain = TransformChain(
    input_variables=["code"],
    output_variables=["rules"],
    transform=transform_retrieval
)

# Step 2 — Analysis LLMChain
analysis_template = PromptTemplate.from_template(
    """
Analyze the Java code for Clean Architecture violations using the given rules.
[Rules]
{rules}

[Code]
{code}

Return:
- Violations (bullet points)
- Referenced rules (bullet points)
- Fix recommendation (1-2 sentences)
"""
)

llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)
analysis_chain = LLMChain(llm=llm, prompt=analysis_template, output_key="analysis")

# Step 3 — SequentialChain
workflow = SequentialChain(
    # TODO: YOUR CODE HERE (fill the three arguments below)
    chains=[],           # [retrieval_chain, analysis_chain]
    input_variables=[],  # ["code"]
    output_variables=[], # ["analysis"]
    verbose=True
)



**Model solution (reference implementation):**


In [ ]:

def transform_retrieval(inputs: dict) -> dict:
    return {"rules": retrieve_relevant_rules(inputs["code"])}

retrieval_chain = TransformChain(
    input_variables=["code"],
    output_variables=["rules"],
    transform=transform_retrieval
)

analysis_template = PromptTemplate.from_template(
    """
Analyze the Java code for Clean Architecture violations using the given rules.
[Rules]
{rules}

[Code]
{code}

Return:
- Violations (bullet points)
- Referenced rules (bullet points)
- Fix recommendation (1-2 sentences)
"""
)

llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)
analysis_chain = LLMChain(llm=llm, prompt=analysis_template, output_key="analysis")

workflow = SequentialChain(
    chains=[retrieval_chain, analysis_chain],
    input_variables=["code"],
    output_variables=["analysis"],
    verbose=True
)



**Expected output (roughly):**
- Mentions *business logic in repository*
- References repo rule: *repositories handle persistence only*
- Suggests moving logic to a service/domain

Now run the usage check:


In [ ]:

test_code = LEAKY_SAMPLES['order_repository']
result = workflow({"code": test_code})
print("=== Workflow analysis (repository) ===\n")
print(result["analysis"] if isinstance(result, dict) else result)



---

## Section 3 — Agents (ReAct with a Retrieval Tool)



### Exercise 3.1 — Create an Agent Tool

**Your Task:** Wrap the retrieval function as an agent tool and run a ReAct agent that can call it autonomously.

**What you need to implement:**
1. Define `Tool` with name, function, and description (names provided)  
2. Initialize a ReAct agent with the tool  
3. Provide the agent with a short input containing the Java code

**Success Criteria:**
- Agent calls the tool during execution (visible in verbose logs)
- Agent provides a final analysis


In [ ]:

from langchain.agents import Tool, initialize_agent, AgentType

# Step 1 — Define tool (names provided)
retrieval_tool = Tool(
    name="clean_arch_retrieval",
    func=retrieve_relevant_rules,
    description="Retrieve Clean Architecture rules relevant to the given text query."
)

# Step 2 — Initialize agent
agent_llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)
agent = initialize_agent(
    tools=[retrieval_tool],
    llm=agent_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

# Step 3 — Agent input template (provided)
agent_prompt = """
Analyze the following Java code for Clean Architecture violations.
If needed, use the tool 'clean_arch_retrieval' with a suitable query.
Provide:
- Violations (bullet points)
- Referenced rules (bullet points)
- Fix recommendation (1-2 sentences)

[Code]
{code}
"""

# Usage check scaffold (run after filling above if needed)
# result = agent.run(agent_prompt.format(code=LEAKY_SAMPLES['order_entity']))
# print(result)



**Model solution (reference implementation):**


In [ ]:

result = agent.run(agent_prompt.format(code=LEAKY_SAMPLES['order_entity']))
print("=== Agent final answer (entity) ===\n")
print(result)



**Expected output (roughly):**
- Mentions that the entity calls a repository/does IO (violation)
- References rule: *entities must not perform IO or depend on infrastructure*
- Suggests moving IO to application/service layer

Note: In the verbose logs above, check that the agent used the retrieval tool at least once.



---

## Section 4 — Model Context Protocol (MCP) via HTTP (Optional)

If you have a local MCP server exposing a `tools/call` endpoint, this exercise shows a minimal async client structure.



### Exercise 4.1 — Implement HTTP call to MCP server (SSE/JSON-RPC)

**Your Task:**
- Build the JSON-RPC request structure
- Send a POST with `aiohttp`
- Parse the JSON response or surface a meaningful connection error

**Success Criteria:**
- Connects if server is running
- Otherwise prints a clear notice and confirms structure is correct


In [ ]:

import aiohttp
import asyncio

async def call_mcp_server(samples_dict: dict, mcp_url: str = "http://127.0.0.1:8765/sse"):
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "tools/call",
        "params": {
            "name": "fix_from_json",
            "arguments": samples_dict
        }
    }
    async with aiohttp.ClientSession() as session:
        async with session.post(mcp_url, json=payload) as resp:
            resp.raise_for_status()
            return await resp.json()

# Usage check (will succeed only if MCP server is available)
async def _try_mcp():
    try:
        result = await call_mcp_server({"test": "sample code"})
        print("✓ MCP connection successful!")
        print(json.dumps(result, indent=2)[:800])
    except aiohttp.ClientConnectorError:
        print("⚠ MCP server not running (expected for demo) — structure OK.")
    except Exception as e:
        print("⚠ MCP call error:", repr(e))

# To run in notebooks:
# await _try_mcp()



---

### You are done 🎉
- You implemented semantic retrieval, RAG‑based analysis, a LangChain SequentialChain, and an Agent with a retrieval tool.
- Reuse these patterns on larger codebases and richer rule sets.
